In [1]:
import pandas as pd

# Specify the file path
file_path1 = 'data_parsing/DSA4266Processed_m6alabel_datainfo.csv'
file_path2 = 'data_parsing/Parsed_Data0.csv'


# Load the CSV file into a DataFrame
m6a_label_df = pd.read_csv(file_path1)
direct_RNASeq_data = pd.read_csv(file_path2)

m6a_label_df['transcript_position'] = pd.to_numeric(m6a_label_df['transcript_position'], errors='coerce')
m6a_label_df['label'] = pd.to_numeric(m6a_label_df['label'], errors='coerce')

In [2]:
import pandas as pd

# Specify the file path
file_path2 = 'data_parsing/Parsed_Data0.csv'

direct_RNASeq_data = pd.read_csv(file_path2)

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# loading and merging data]

merged_df = pd.merge(direct_RNASeq_data, m6a_label_df, 
                     left_on=['transcript_id', 'position'], 
                     right_on=['transcript_id', 'transcript_position'])
# Initialize label encoder
label_encoder = LabelEncoder()

In [4]:
# Function to extract the specific sequence
def extract_specific_sequence(row):
    sequence_columns = [col for col in row.index if col.startswith('sequence_') and row[col] == 1]
    if sequence_columns:
        return sequence_columns[0].split('sequence_')[1]
    else:
        return None

# Apply the function to each row
merged_df['specific_sequence'] = merged_df.apply(extract_specific_sequence, axis=1)


merged_df['1st_pos'] = merged_df['specific_sequence'].apply(lambda x: x[0])
merged_df['2nd_pos'] = merged_df['specific_sequence'].apply(lambda x: x[1])
merged_df['3rd_pos'] = merged_df['specific_sequence'].apply(lambda x: x[2])
merged_df['4th_pos'] = merged_df['specific_sequence'].apply(lambda x: x[3])
merged_df['5th_pos'] = merged_df['specific_sequence'].apply(lambda x: x[4])

merged_df.head()


seq_col = [col for col in merged_df.columns.tolist() if col.startswith('sequence_')]
merged_df.drop(seq_col, axis=1, inplace=True)
merged_df.drop('specific_sequence', axis=1, inplace=True)
merged_df['1st_pos'] = merged_df['1st_pos'].astype('category')
merged_df['2nd_pos'] = merged_df['2nd_pos'].astype('category')
merged_df['3rd_pos'] = merged_df['3rd_pos'].astype('category')
merged_df['4th_pos'] = merged_df['4th_pos'].astype('category')
merged_df['5th_pos'] = merged_df['5th_pos'].astype('category')

In [5]:
from sklearn.model_selection import train_test_split
# Get unique gene IDs
unique_gene_ids = merged_df['gene_id'].unique()

# Split unique gene IDs into training and test groups
train_gene_ids, test_gene_ids = train_test_split(unique_gene_ids, test_size=0.2, random_state=42)

# Create training and test datasets
train_data = merged_df[merged_df['gene_id'].isin(train_gene_ids)]
test_data = merged_df[merged_df['gene_id'].isin(test_gene_ids)]

In [6]:
# Split the data into label 0 and label 1
label_0_data = train_data[train_data['label'] == 0]
label_1_data = train_data[train_data['label'] == 1]

# Calculate the number of label 1 samples needed to achieve a ratio
desired_ratio = 1 / 1
num_label_0 = len(label_0_data)
num_label_1_needed = int(num_label_0 * desired_ratio)

# Oversample label 1 data to match the desired ratio
label_1_oversampled = pd.concat([label_1_data] * (num_label_1_needed // len(label_1_data)), ignore_index=True)

# Combine oversampled label 1 data with original label 0 data
train_data = pd.concat([label_1_oversampled, label_0_data])

# Shuffle the data to randomize the order
train_data = train_data.sample(frac=1, random_state=42)

In [7]:
grouped_train = train_data.groupby("gene_id")

# Define a function for standardization
def standardize_column_mean(column):
    return (column - column.mean()) / column.std()

def standardize_column_sd(column):
    return column / column.std()

# Apply the standardization function to the "0 mean" column within each group
train_data["0 mean standardized"] = grouped_train["0 mean"].transform(standardize_column_mean)
train_data["0 sd standardized"] = grouped_train["0 sd"].transform(standardize_column_sd)
train_data["0 length standardized"] = grouped_train["0 length"].transform(standardize_column_mean)

In [8]:
grouped_test = test_data.groupby("gene_id")

# Apply the standardization function to the "0 mean" column within each group
test_data["0 mean standardized"] = grouped_test["0 mean"].transform(standardize_column_mean)
test_data["0 sd standardized"] = grouped_test["0 sd"].transform(standardize_column_sd)
test_data["0 length standardized"] = grouped_test["0 length"].transform(standardize_column_mean)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [9]:
final_train_dataset = train_data.drop(columns=['transcript_id','position','transcript_position','gene_id','0 length', '0 sd', '0 mean', '0 min', '0 max'])
final_test_dataset = test_data.drop(columns=['transcript_id','position','transcript_position','gene_id','0 length', '0 sd', '0 mean', '0 min', '0 max'])

In [10]:
# Perform one-hot encoding for categorical columns
final_train_dataset = pd.get_dummies(final_train_dataset, columns=['1st_pos', '2nd_pos', '3rd_pos', '4th_pos', '5th_pos'])

In [11]:
# Perform one-hot encoding for categorical columns
final_test_dataset = pd.get_dummies(final_test_dataset, columns=['1st_pos', '2nd_pos', '3rd_pos', '4th_pos', '5th_pos'])

In [12]:
final_train_dataset['y_label'] = final_train_dataset['label']
final_train_dataset = final_train_dataset.drop(columns='label')

In [13]:
final_test_dataset['y_label'] = final_test_dataset['label']
final_test_dataset = final_test_dataset.drop(columns='label')

In [14]:
import tensorflow as tf
from tensorflow import keras

2023-11-09 11:42:26.116523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 11:42:26.207486: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-09 11:42:26.207508: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-09 11:42:26.756394: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [15]:
X_test = final_test_dataset.drop(columns=['y_label'])# Input features 
y_test = final_test_dataset['y_label'] # Target labels

In [17]:
num_features = final_train_dataset.shape[1]-1

model = keras.Sequential([
    keras.layers.Input(shape=(num_features,)),  # num_features should be the number of features in your data
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'), 
    keras.layers.Dense(1, activation='sigmoid')  # Assuming binary classification
])

X = final_train_dataset.drop(columns=['y_label'])# Input features 
y = final_train_dataset['y_label'] # Target labels

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X, y,  epochs=5, batch_size=256)

Epoch 1/5


2023-11-09 11:47:09.032190: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2214978144 exceeds 10% of free system memory.


63620/63620 [==============================] - 55s 859us/step - loss: 0.4916 - accuracy: 0.7562
Epoch 2/5
63620/63620 [==============================] - 55s 858us/step - loss: 0.4889 - accuracy: 0.7572
Epoch 3/5
63620/63620 [==============================] - 55s 859us/step - loss: 0.4883 - accuracy: 0.7575
Epoch 4/5
63620/63620 [==============================] - 55s 858us/step - loss: 0.4880 - accuracy: 0.7577
Epoch 5/5
63620/63620 [==============================] - 55s 864us/step - loss: 0.4878 - accuracy: 0.7578


In [18]:
evaluation = model.evaluate(X_test, y_test)
predictions = (model.predict(X_test) >= 0.5).astype(int)

72217/72217 [==============================] - 39s 539us/step


In [19]:
model.save('model_NN_final.h5')

In [20]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, predictions))

Accuracy: 74.00%
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.74      0.84   2209788
         1.0       0.12      0.74      0.20    101131

    accuracy                           0.74   2310919
   macro avg       0.55      0.74      0.52   2310919
weighted avg       0.95      0.74      0.82   2310919

